In [ ]:
# Using Pubsub with Cloud Run 
# TASK 3 Step 2
#Create a push subscriber running in Cloud Run or Cloud Functions. 
#Process the messages as they come in. Parse them, and write the data to BigQuery where it can be analyzed. 
#In Looker Studio, create a simple report that shows clicks by page in real time.

import os
import random
from datetime import datetime
import json
from google.cloud import pubsub_v1
from google.api_core.exceptions import AlreadyExists


In [29]:
# Project, topic, and subscription variables
PROJECT_ID = 'jellyfish-training-demo-6'
REGION     = 'us-central1'
TOPIC_ID = 'projects/jellyfish-training-demo-6/topics/lab_topic'
SUBSCRIPTION_ID = 'push_cloudrun'
dataset_id = "dsl_project"  # Replace with your dataset ID
table_id = "website-visits_batch"  # Replace with your table ID

! gcloud config set run/region REGION

Updated property [run/region].


In [8]:
# using existing repository in artifact registry: 
# us-central1-docker.pkg.dev/jellyfish-training-demo-6/cloud-run-source-deploy
REPOSITORY = 'cloud-run-source-deploy'
# create a new one if needed, 
#! gcloud artifacts repositories create dsl-repo --repository-format=docker --location=REGION


In [ ]:
# Build your container and publish on Artifact Registry:
### Create a Pub/Sub subscription 

! gcloud builds submit --tag us-central1-docker.pkg.dev/jellyfish-training-demo-6/cloud-run-source-deploy/pubsub

In [ ]:
#Run the following command to deploy your app:

!gcloud run deploy pubsub-cloudrun-push --quiet --region=$REGION --image us-central1-docker.pkg.dev/jellyfish-training-demo-6/cloud-run-source-deploy/pubsub --no-allow-unauthenticated

In [42]:
# Create a Pub/Sub subscription 
# endpoint from the deployment: https://pubsub-cloudrun-push-387528315161.us-central1.run.app

! gcloud pubsub subscriptions create $SUBSCRIPTION_ID --topic='projects/jellyfish-training-demo-6/topics/lab_topic' \
--ack-deadline=600 \
--push-endpoint='https://pubsub-cloudrun-push-387528315161.us-central1.run.app'/ \
--push-auth-service-account=387528315161-compute@developer.gserviceaccount.com

Created subscription [projects/jellyfish-training-demo-6/subscriptions/push_cloudrun].


In [43]:
# To test the end-to-end solution:
# Send a Pub/Sub message to the topic:

!gcloud pubsub topics publish lab_topic --message "Runner"

messageIds:
- '14589122503927140'
